In [22]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib as mp
from selenium import webdriver

In [25]:
def drug_scrape(url,file_name):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    table = soup.find_all('table')[0]
    headers = []
    for th in table.find_all('th'):
        headers.append(th.get_text())

    names = []
    indic = []
    types = []
    rev = []
    num_rev = []
    urls = []
    urls_all = []
    for tr in table.tbody.find_all('tr'):
        td = tr.find_all('td')
        #creating list of all drugs
        names.append(td[0].a.text)
        # scraping on or off label information
        indic.append(td[1].text)
        # scrape OTC or RX information
        types.append(td[2].text)
        # scraping number of reviews of drug
        rev.append(td[3].a.text)
        # scraping all drug review page urls
        urls_all.append(td[3].a['href'])
        # scraping the urls of drugs with 100 reviews or more
        a = td[3].a.text.split()
        if int(a[0])>= 100:
            urls.append((td[3].a['href']))
    #get number of reviews as just the number
    for i in rev:
        a = i.split()
        num_rev.append(int(a[0])) 
    
    
# drug delivery method 
    header = []
    for ur in urls_all:
        url = "https://www.webmd.com" + ur
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'html.parser')
        heading = soup.find('div', class_= 'tb_main')
        header.append(heading.h1.text)
    drug_method =[]
    for h in header:
        a = h.split()
        drug_method.append(a[-1]) 
    
    rates = get_ratings(urls)
        
    drug_df = pd.DataFrame({'drug': pd.Series(names),
                            'indication': pd.Series(indic),
                            'type of drug' : pd.Series(types),
                            'drug delivery method' : pd.Series(drug_method),
                            'reviews': pd.Series(num_rev),
                           'effectiveness score' : pd.Series(rates[0]),
                           'ease of use score' : pd.Series(rates[1]),
                           'satisfaction score' : pd.Series(rates[2])})
    
    drug_df.to_csv(file_name,index=False)

    

#Using selenium to scrape star ratings for each drug with  more than 100 reviews
def get_ratings(urls):
    effective_val =[]
    ease_of_use_val = []
    satisfac_val = []
    chromedriver = "/Users/danielle/anaconda3/bin/chromedriver"
    for ur in urls:
        site = "https://www.webmd.com" + ur
        browser = webdriver.Chrome(chromedriver)
        wmd = browser.get(site)
        ratings = browser.find_elements_by_class_name('numbRating')
        effective_val.append(float(ratings[0].text.strip('()')))
        ease_of_use_val.append(float(ratings[1].text.strip('()')))
        satisfac_val.append(float(ratings[2].text.strip('()')))
        browser.close()
    return [effective_val, ease_of_use_val, satisfac_val]

           



    

In [26]:
drugs = ["https://www.webmd.com/drugs/2/condition-967/anxiety", 
         "https://www.webmd.com/drugs/2/condition-1028/attention-deficit%20hyperactivity%20disorder",
         "https://www.webmd.com/drugs/2/condition-1022/depression",
         "https://www.webmd.com/drugs/2/condition-3063/insomnia"
        ]
file_names = ['anxiety.csv','ADHD.csv','depression.csv','insomnia.csv']

In [ ]:
for i in range(len(drugs)):
    drug_scrape(drugs[i],file_names[i])